In [3]:
import torch
import lib.utils.bookkeeping as bookkeeping
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lib.models.models as models
import lib.models.model_utils as model_utils
from lib.datasets import mnist, maze, protein, synthetic
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
from lib.d3pm import make_diffusion
import os
from lib.datasets.maze import maze_acc


/Users/paulheller/PythonRepositories/d3pm/d3pmvenv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [4]:
# creating path
path = "SavedModels/MNIST/"
date = '2024-05-28' # 2
config_name = 'config_001.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_1.pt'

config_path = os.path.join(path, date, config_name)
checkpoint_path = os.path.join(path, date, model_name)

In [5]:
# creating models
cfg = bookkeeping.load_config(config_path)

diffusion = make_diffusion(cfg.model)
#print(cfg)
device = torch.device(cfg.device)
print(device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

optimizer = torch.optim.Adam(model.parameters(), cfg.optimizer.lr)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}
state = bookkeeping.load_state(state, checkpoint_path, device)
state['model'].eval()


2024-05-28 16:36:26.354 | INFO     | lib.d3pm:__init__:100 - [compute transition matrix]: gaussian


in betas 1000
from beta 1000


2024-05-28 16:38:43.965 | INFO     | lib.d3pm:__init__:121 - [trainsition matrix]: torch.Size([1000, 256, 256])
2024-05-28 16:38:43.977 | INFO     | lib.d3pm:__init__:124 - [Construct transition matrices for q(x_t|x_start)]
2024-05-28 16:39:02.817 | INFO     | lib.d3pm:__init__:144 - [tilde(Q)t]: torch.Size([1000, 256, 256])


cpu
number of parameters:  1013024
ema state dict function


In [6]:
n_samples = 4
if cfg.data.name == 'Maze3S':
    shape = (n_samples, 1, 15, 15)
elif cfg.data.name == 'DiscreteMNIST':
    shape = (n_samples, 1, 28, 28)
elif cfg.data.name == 'SyntheticData':
    shape = (n_samples, 32)
else:
    raise ValueError("wrong")

samples = diffusion.p_sample_loop(state['model'], shape, cfg.model.num_timesteps).cpu().numpy()
saved_samples = samples

2024-05-28 16:39:03.714 | INFO     | lib.d3pm:p_sample_loop:553 - cpu
1000it [23:46,  1.43s/it]


In [ ]:
from lib.datasets.mnist_fid import evaluate_fid_score
data = np.load(f'sample_path.npy')
dataset_location = "lib/datasets"
fid_values = []
cfg.data.train = False
dataset = dataset_utils.get_dataset(cfg, device, dataset_location)

dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=data.shape[0],
    shuffle=cfg.data.shuffle)
for true_data in (dataloader):
    #print(f'mnist_hollow_{sampler_n}{step}.npy')
    print("----------------------------------")

    fid = evaluate_fid_score(data, true_data.cpu().numpy(), 100)
    print("FID:", fid)
    fid_values.append(fid)
    break
print(fid_values)


In [7]:
is_img = True
if is_img:
    samples = samples.reshape(-1, 1, cfg.data.image_size, cfg.data.image_size)
    saving_train_path = os.path.join(cfg.saving.sample_plot_path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
    fig = plt.figure(figsize=(9, 9)) 
    for i in range(n_samples):
        plt.subplot(int(np.sqrt(n_samples)), int(np.sqrt(n_samples)), 1 + i)
        plt.axis("off")
        plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")
 
    # saving_train_path
    plt.savefig('image_samples.pdf', transparent=True)
    plt.show()
    plt.close()
else:
    bm, inv_bm = synthetic.get_binmap(cfg.model.concat_dim, cfg.data.binmode)
    print(inv_bm)
    samples = synthetic.bin2float(samples.astype(np.int32), inv_bm, cfg.model.concat_dim, cfg.data.int_scale)

 
    saving_plot_path = os.path.join(path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
    saving_np_path = os.path.join(path, f"samples_{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.npy")
    synthetic.plot_samples(samples, 'synthetic_samples.pdf', im_size=cfg.data.plot_size, im_fmt="pdf")

AttributeError: "'sampler'"

In [ ]:
correct_mazes = maze_acc(saved_samples)

In [ ]:
cfg.data.name = 'Maze3SComplete'
cfg.data.batch_size = n_samples

if cfg.data.name == 'Maze3SComplete':
    limit = cfg.data.batch_size
    cfg.data.limit = limit 

dataset = dataset_utils.get_dataset(cfg, device)
dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=cfg.data.batch_size,
    shuffle=cfg.data.shuffle)

for i in dataloader:
    true_dl = i
    c_i = maze_acc(i.cpu().numpy())
    true_dl = true_dl.reshape(cfg.data.batch_size, -1) #.flatten()